In [1]:
from gnomad_db.database import gnomAD_DB
import pandas as pd
import numpy as np

# Download SQLite preprocessed files

I have preprocessed and created sqlite3 files for gnomAD v2, v3, v4 for you, which can be easily downloaded from here. They contain all variants on the 24 standard chromosomes.

You can find the links to download the sqlite3 file in the README (https://github.com/KalinNonchev/gnomAD_DB)

In [8]:
# uncomment if you actually want to download it
download_link = "https://zenodo.org/records/10066310/files/gnomad_db_wes_v4.0.sqlite3.gz?download=1"
output_dir = "test_dir" # database_location
gnomAD_DB.download_and_unzip(download_link, output_dir) 

Starting downloading...


gnomad_db_wes_v4.0.sqlite3.gz?download=1: 7.31GB [12:40, 9.62MB/s]                                                                                                          


Starting unzipping. This can take some time...
Database location: test_dir/gnomad_db.sqlite3
Done!


# Initialize Database

In [9]:
# pass dir
database_location = "test_dir"

In [11]:
# initialize database
db = gnomAD_DB(database_location, gnomad_version="v45")

AssertionError: We don't support this version: v45. Please select one fo the following ones: ['v2', 'v3', 'v4']

# Insert gnomAD variants into the database from single tsv file
Look into insertVariants notebook to do it for big vcf files

In [ ]:
# get some variants
var_df = pd.read_csv("data/test_vcf_gnomad_chr21_10000.tsv.gz", sep="\t", names=db.columns, index_col=False)
# preprocess missing values
# IMPORTANT: The database removes internally chr prefix (chr1->1)
var_df = var_df.replace(".", np.NaN)
var_df.head()

In [ ]:
# insert variants
db.insert_variants(var_df)

# Query MAF

In [ ]:
# check db columns, which we can query
db.columns

In [ ]:
var_df = var_df[["chrom", "pos", "ref", "alt"]]
var_df.head()

## You can pass a dataframe with variants
It should contain the columns: [chrom, pos, ref, alt]

In [ ]:
db.get_info_from_df(var_df, "AF").head() # only one columns

In [ ]:
db.get_info_from_df(var_df, "AF, AF_popmax").head() # multiple columns

In [ ]:
db.get_info_from_df(var_df, "*") # everything

In [ ]:
dummy_var_df = pd.DataFrame({
    "chrom": ["1", "21"], 
    "pos": [21, 9825790], 
    "ref": ["T", "C"], 
    "alt": ["G", "T"]})
dummy_var_df

In [ ]:
db.get_info_from_df(dummy_var_df, "AF")

## You can pass a single string as a variant

In [7]:
db.get_info_from_str("10:95606780:A>C", "*")

chrom                   10
pos               95606780
ref                      A
alt                      C
filter                PASS
AC                     2.0
AN                628768.0
AF                0.000003
MQ                    60.0
QD                 12.1796
ReadPosRankSum       0.365
VarDP                412.0
AS_VQSLOD           5.5239
AC_grpmax              2.0
AN_grpmax         350092.0
AF_grpmax         0.000006
AF_eas                 0.0
AF_nfe            0.000006
AF_fin                 0.0
AF_afr                 0.0
AF_asj                 0.0
Name: 0, dtype: object

In [ ]:
db.get_info_from_str("21:9825790:C>T", "*")

## You can look for the MAF scores in an interval

In [ ]:
db.get_info_for_interval(chrom=21, interval_start=9825780, interval_end=9825799, query="*")